In [11]:
from ptl import get

In [12]:
get.setup(debug=True, driver=True)

> PLATFORM:	win32
> DEBUG:	True
> LIB_PATH:	E:\jupyter_notebook\20171106\ptl\
> ROOT_PATH:	E:\jupyter_notebook\20171106
> START_TIME:	20171106_1005
> Open new webdriver.


In [13]:
domain = 'reddit.com'
overview = 'https://www.spyfu.com/overview/domain?query='
seo_cp = 'https://www.spyfu.com/seo/competitors/domain?rowsPerPage=2500&query='
ppc_cp = 'https://www.spyfu.com/ppc/competitors/domain?rowsPerPage=2500&query='
seo_kw = 'https://www.spyfu.com/seo/keywords/domain?rowsPerPage=2500&query='
ppc_kw = 'https://www.spyfu.com/ppc/keywords/domain?rowsPerPage=2500&query='

In [14]:
get.soup(overview+domain, get.DR)

> Driver get: https://www.spyfu.com/overview/domain?query=reddit.com


In [15]:
get.bef('organic-search-widget-keywords-count')

1722746.0

In [16]:
get.bef('organic-search-widget-monthly-clicks-count')

129199999.99999999

In [17]:
get.bef('paid-search-widget-keywords-count')

209

In [18]:
get.bef('paid-search-widget-monthly-clicks-count')

0

In [19]:
get.bet('domain-overview-inbound-clicks-ratio-percent')

'100%'

In [59]:
def get_table(index=0):
    tbl = get.be('backgrid','table',index=index)
    if not tbl:
        get.log("> There is no table data")
        return
    cols = [e.text.strip() for e in tbl.thead.findAll('th')][1:] #column names
    #get.log('> Columns: '+str(cols))
    rows = []
    for tr in [tr for tr in tbl.tbody.findAll('tr')]:
        try:
            row = {}
            tds = tr.findAll('td')[1:]
            for  i, td in enumerate(tds):
                if cols[i]=='Domain Name': #domain name column
                    row[cols[i]] = td.text
                elif cols[i]=='Overlap': 
                    if td.a: #overlap column
                        overlap = td.a.div['style'].replace('width: ', '').replace('%', '')
                        row[cols[i]] = float(overlap)
                    else:
                        if td.i: # rank change column
                            text = td.text.split('(')
                            row['Rank']= text[0]
                            row['Rank Change']= text[1].replace(')', '') if len(text)>1 else ''
                        else:
                            row[cols[i]]=-1               
                elif cols[i]=='Keyword':
                    keyword = get.bec(td)
                    row[cols[i]] = keyword[0].text
                    row[cols[i]+' URL'] = keyword[1].text if len(keyword)>1 else ''
                elif cols[i]=='Ad Timeline':
                    keyword = get.bec(td)
                    row[cols[i]] = keyword[0].text                    
                elif len(cols[i])>0: #other column
                    if '(' in td.text:
                        text=td.text.split('(')
                        row['Rank'] = int(text[0])
                        text[1] = text[1].replace(')', '')
                        row[cols[i]]= int(text[1]) if text[1].isdigit() else 0
                    else: row[cols[i]] = get.s2f(td.text) 
        except: get.pe()
        rows.append(row)
    return get.DataFrame(rows)

In [21]:
get.WDW_DELAY=120

In [22]:
get.soup(seo_cp+domain, get.DR, waitc='class', waitv='backgrid')

> Driver get: https://www.spyfu.com/seo/competitors/domain?rowsPerPage=2500&query=reddit.com
> Wait for element:	class=backgrid


In [23]:
df = get_table(0)
get.save(df, 'tables/'+domain+'_seo_competitors')
df.head()

,CommonKeywords,Domain Name,MonthlyClicks,MonthlyValue of Clicks,Number ofKeywords,Overlap
0,-9999.0,reddit.com,1.290000e+08,6.140000e+07,1720000.0,-1.000000
1,1050000.0,youtube.com,2.380000e+12,1.410000e+12,10900000.0,91.000000
2,339000.0,quora.com,5.250000e+07,3.780000e+07,1680000.0,67.557453
3,687000.0,amazon.com,4.910000e+08,2.710000e+08,7760000.0,51.619269
4,620000.0,wikipedia.org,3.680000e+12,2.900000e+12,7690000.0,47.812065


In [24]:
get.soup(ppc_cp+domain, get.DR, waitc='class', waitv='backgrid')

> Driver get: https://www.spyfu.com/ppc/competitors/domain?rowsPerPage=2500&query=reddit.com
> Wait for element:	class=backgrid


In [25]:
df = get_table(0)
get.save(df, 'tables/'+domain+'_paid_competitors')
df.head()

,CommonKeywords,Domain Name,MonthlyAd Budget,MonthlyPaid Clicks,MonthlyPaid Keywords,Overlap
0,-9999,reddit.com,-9999.0,-9999.0,-9999.0,-1.000000
1,1,easypii.com,-9999.0,-9999.0,-9999.0,91.000000
2,1,funnel.io,8910.0,1880.0,479.0,48.025611
3,1,rm-ransomwarerecovery.com,101.0,79.3,9.0,41.708333
4,3,imonitorsoft.com,5510.0,2160.0,337.0,23.552941


In [46]:
get.soup(seo_kw+domain, get.DR, waitc='class', waitv='backgrid')

> Driver get: https://www.spyfu.com/seo/keywords/domain?rowsPerPage=2500&query=reddit.com
> Wait for element:	class=backgrid


In [60]:
df = get_table(1)
get.save(df, 'tables/'+domain+'_seo_keywords')
df.head()

,Cost/Click,Est. Click Value/Mo,Est. Clicks/Mo,Keyword,Keyword URL,Rank,Rank (Change),Ranking Difficulty,Searches/Mo
0,9.0,2570000.0,262000.0,googe.com,reddit.com/r/todayilearned/comments/l00v9/til...,4,4,63,6120000.0
1,13.0,2410000.0,177000.0,googlle.com,reddit.com/domain/googlle.com,6,3,66,6120000.0
2,16.0,1560000.0,95000.0,twitt er,reddit.com/r/neuroscience/comments/4bf1pi/wha...,21,10,63,11100000.0
3,12.0,1030000.0,80100.0,www.restraunts.com,reddit.com/r/freebies/comments/6bgj0y/tmobile...,11,13,47,5000000.0
4,48.0,998000.0,20600.0,cars insurence,reddit.com/r/cars/comments/4o7c84/need_cheap_...,2,3,59,246000.0


In [63]:
df['Est. Click Value/Mo']

0       2570000.0
1       2410000.0
2       1560000.0
3       1030000.0
4        998000.0
5        841000.0
6        638000.0
7        482000.0
8        375000.0
9        361000.0
10       342000.0
11       339000.0
12       318000.0
13       271000.0
14       269000.0
15       265000.0
16       257000.0
17       255000.0
18       244000.0
19       198000.0
20       197000.0
21       153000.0
22       148000.0
23       142000.0
24       132000.0
25       124000.0
26       119000.0
27       113000.0
28       106000.0
29       100000.0
          ...    
2470       1230.0
2471       1230.0
2472       1230.0
2473       1230.0
2474       1230.0
2475       1230.0
2476       1230.0
2477       1230.0
2478       1230.0
2479       1230.0
2480       1230.0
2481       1230.0
2482       1220.0
2483       1220.0
2484       1220.0
2485       1220.0
2486       1220.0
2487       1220.0
2488       1220.0
2489       1220.0
2490       1220.0
2491       1220.0
2492       1220.0
2493       1220.0
2494      

In [28]:
get.soup(ppc_kw+domain, get.DR, waitc='class', waitv='backgrid')

> Driver get: https://www.spyfu.com/ppc/keywords/domain?rowsPerPage=2500&query=reddit.com
> Wait for element:	class=backgrid


In [45]:
df = get_table(0)
get.save(df, 'tables/'+domain+'_paid_keywords')
df.head()

,Ad Position,Ad Timeline,Cost Per Click,Keyword,Keyword URL,Monthly Cost,Monthly Searches,Total Ads
0,7,,5.0,day trading how to,reddit.com,1540.0,30.0,10
1,5,,5.0,how to invest your money,reddit.com,1420.0,4380.0,10
2,7,,5.0,best day trading,reddit.com,370.0,150.0,11
3,3,,6.0,best stock charts,reddit.com,350.0,570.0,7
4,1,,0.0,download stock data,reddit.com,210.0,300.0,1


[<div class="keyword"><a class="goto-keyword" href="/keyword/overview?query=day%20trading%20how%20to">day trading how to</a></div>, <div class="link"><a href="http://reddit.com" target="_blank" title="reddit.com"><i class="fa fa-external-link"></i> </a><a class="goto-url" href="/overview/url?query=reddit.com">reddit.com</a></div>]
[<div class="keyword"><a class="goto-keyword" href="/keyword/overview?query=how%20to%20invest%20your%20money">how to invest your money</a></div>, <div class="link"><a href="http://reddit.com" target="_blank" title="reddit.com"><i class="fa fa-external-link"></i> </a><a class="goto-url" href="/overview/url?query=reddit.com">reddit.com</a></div>]
[<div class="keyword"><a class="goto-keyword" href="/keyword/overview?query=best%20day%20trading">best day trading</a></div>, <div class="link"><a href="http://reddit.com" target="_blank" title="reddit.com"><i class="fa fa-external-link"></i> </a><a class="goto-url" href="/overview/url?query=reddit.com">reddit.com</a><